In [1]:
import mlx.core as mx
from einops import einsum, rearrange

# 指标操作与张量基本运算

In [2]:
x = mx.array([1, 3, 5, 7])
x1 = mx.reshape(x, [2, 2])
x1_ = x.reshape(2, 2)
print(x1)
print(x1_)

array([[1, 3],
       [5, 7]], dtype=int32)
array([[1, 3],
       [5, 7]], dtype=int32)


In [3]:
x2 = x1.transpose(1, 0)
print(x2)

array([[1, 5],
       [3, 7]], dtype=int32)


## 向量化与矩阵化的数学公式符号约定:

张量的矩阵化。如果将某几个指标(例如 $i_a,i_b,i_c$) 合并作为左指标,其余合并作为右指标,该矩阵简记为 $T_{[i_a i_b i_c]}$ , 在不引起误解的情况下,剩余的指标可以不写出来,或也可以写在第二个方括号中,记为 $T_{[i_a i_b i_c][\cdots]}$ 。若不希望指定代表各个指标的名字 (字母), 可以写成 $T_{[0,2,\cdots]}$ , 方括号中的数字代表张量的第几个指标。

如果将某一个指标 $i_m$ 作为矩阵左指标, 并合并其余指标作为矩阵右指标,该矩阵简记为 $T_{[i_m]}$ ; 如果将前 $m$ 个指标合并作为左指标,剩下的指标合并作为右指标,该矩阵简记为 $T_{[i_0 \cdots i_{m-1}][i_m \cdots i_{N-1}]} $ 或 $T_{[i_0 \cdots i_{m-1}][\cdots]} $ 或 $T_{[0 \cdots m-1]}$. 如果需要将一个张量的所有指标合并成一个指标,则以此获得的向量可简记为 $T_{[:]}$ , 其也被称为 $T$ 的向量化 (vectorization)

## 外积


外积运算: $U = X \otimes Y \leftrightarrow U_{abc...ijk...} = X_{abc...}Y_{ijk...}$

注意左张量的维度在前，右张量的维度在后

In [4]:
x = mx.array([1, 3, 5])
y = mx.array([2, 4, 6, 8])
# 得到张量的形状是 (3, 4)
z1 = mx.outer(x, y)
z2 = mx.einsum("a,b -> ab", x, y)
print(z1)
print(z2)

# equivalently
z3 = x.reshape(3, 1) * y.reshape(1, 4)
print(z3)

array([[2, 4, 6, 8],
       [6, 12, 18, 24],
       [10, 20, 30, 40]], dtype=int32)
array([[2, 4, 6, 8],
       [6, 12, 18, 24],
       [10, 20, 30, 40]], dtype=int32)
array([[2, 4, 6, 8],
       [6, 12, 18, 24],
       [10, 20, 30, 40]], dtype=int32)


### Einsum

*   其输入为字符串公式与相关张量；
*   公式包含箭头 “->”，箭头左侧为各个待收缩张量的指标，右侧为收缩所得张量的指标；
*   左侧各个张量的指标用逗号隔开，共有指标使用同一个字母表示；
*   当左侧出现的指标没有出现在右侧时，说明对该指标作求和运算；

例子1:
 `torch.einsum("a,b -> ab", x, y)`
 从outer对应的einsum可以容易看出outer的计算公式为 $w_{ab} = u_a v_b$。


In [5]:
# 例子2
z = mx.einsum("a,b -> ab", x, y)
# sum over the first index
z_sum = mx.einsum("ab -> a", z)
print(z_sum)

array([20, 60, 100], dtype=int32)


### Kronecker Product

Einsum 公式:
$u_av_b = w_{ab} \rightarrow w_{[ab]}$

In [6]:
u = mx.array([1, 3, 5])
v = mx.array([2, 4, 6, 8])

kron1 = mx.kron(u, v)
kron2 = mx.einsum("a,b -> ab", u, v).flatten()
print(kron1)
print(kron2)

array([2, 4, 6, ..., 20, 30, 40], dtype=int32)
array([2, 4, 6, ..., 20, 30, 40], dtype=int32)


In [7]:
a = mx.random.normal([2, 3])
b = mx.random.normal([4, 5])

kron1 = mx.kron(a, b)
kron2 = einsum(a, b, "a b, c d -> a c b d")
kron2 = rearrange(kron2, "a c b d -> (a c) (b d)")

assert mx.allclose(kron1, kron2).item()